# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

In [1]:
!pip install -r requirements.txt

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
tf.__version__

'2.3.1'

### Importar o dataset

In [4]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [6]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [10]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [11]:
ann.add(tf.keras.layers.Dense(units=6, kernel_initializer='random_normal', activation='selu'))

### Adding the second hidden layer

In [12]:
ann.add(tf.keras.layers.Dense(units=6, activation='selu'))
ann.add(tf.keras.layers.Dense(units=6, activation='elu'))

### Adding the output layer

In [13]:
ann.add(tf.keras.layers.Dense(units=1,  activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [14]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [15]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 844us/step - loss: 0.4997 - accuracy: 0.7810
Epoch 2/100
250/250 [==============================] - 0s 772us/step - loss: 0.4277 - accuracy: 0.8080
Epoch 3/100
250/250 [==============================] - 0s 840us/step - loss: 0.4229 - accuracy: 0.8114
Epoch 4/100
250/250 [==============================] - 0s 848us/step - loss: 0.4187 - accuracy: 0.8173
Epoch 5/100
250/250 [==============================] - 0s 836us/step - loss: 0.4136 - accuracy: 0.8278
Epoch 6/100
250/250 [==============================] - 0s 744us/step - loss: 0.4089 - accuracy: 0.8309
Epoch 7/100
250/250 [==============================] - 0s 896us/step - loss: 0.4060 - accuracy: 0.8319
Epoch 8/100
250/250 [==============================] - 0s 916us/step - loss: 0.4015 - accuracy: 0.8345
Epoch 9/100
250/250 [==============================] - 0s 864us/step - loss: 0.3957 - accuracy: 0.8365
Epoch 10/100
250/250 [==============================] - 0s 744us/step - l

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [16]:
y_pred = ann.predict(X_test)
y_pred

array([[0.28459346],
       [0.29276565],
       [0.17109558],
       ...,
       [0.13377559],
       [0.1893861 ],
       [0.19832236]], dtype=float32)

In [17]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [18]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1519   76]
 [ 200  205]]


In [19]:
accuracy = (cm[0,0] + cm[1,1])/cm.sum()
print(accuracy)

0.862


# Part 4 - Avaliando, Melhorando e refinando a ANN


In [20]:
# Avaliando
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

In [21]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu', input_dim = 12))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu'))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'elu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [22]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)

In [23]:
mean = accuracies.mean()

In [24]:
variance = accuracies.std()

In [25]:
print(mean)
print(variance)

0.8158750057220459
0.0133586429953171


In [26]:
# Melhorando ANN
# Dropout Regularization to reduce overfitting if needed
from keras.layers import Dropout

In [27]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'elu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [28]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)

In [29]:
mean = accuracies.mean()

In [30]:
variance = accuracies.std()

In [31]:
print(mean)
print(variance)

0.8139999985694886
0.011508144785641314


In [32]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'elu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [33]:
classifier = KerasClassifier(build_fn = build_classifier)

parameters = {'batch_size': [10, 25, 32],
              'epochs': [50, 50],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)


In [34]:
grid_search = grid_search.fit(X_train, y_train)


=========] - 0s 796us/step - loss: 0.4342 - accuracy: 0.8188
Epoch 7/50
225/225 [==============================] - 0s 800us/step - loss: 0.4332 - accuracy: 0.8174
Epoch 8/50
225/225 [==============================] - 0s 787us/step - loss: 0.4312 - accuracy: 0.8172
Epoch 9/50
225/225 [==============================] - 0s 840us/step - loss: 0.4330 - accuracy: 0.8167
Epoch 10/50
225/225 [==============================] - 0s 831us/step - loss: 0.4308 - accuracy: 0.8182
Epoch 11/50
225/225 [==============================] - 0s 822us/step - loss: 0.4304 - accuracy: 0.8182
Epoch 12/50
225/225 [==============================] - 0s 836us/step - loss: 0.4304 - accuracy: 0.8199
Epoch 13/50
225/225 [==============================] - 0s 769us/step - loss: 0.4292 - accuracy: 0.8213
Epoch 14/50
225/225 [==============================] - 0s 773us/step - loss: 0.4295 - accuracy: 0.8203
Epoch 15/50
225/225 [==============================] - 0s 773us/step - loss: 0.4288 - accuracy: 0.8206
Epoch 16/50
225

In [35]:
best_parameters = grid_search.best_params_

In [36]:
best_accuracy = grid_search.best_score_

In [37]:
print(best_parameters)
print(best_accuracy)

{'batch_size': 10, 'epochs': 50, 'optimizer': 'adam'}
0.835125


> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)